In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import cohen_kappa_score
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:
with open ('abalone19.dat') as f:
     lines = [line.rstrip('\n')for line in f]

In [3]:
len(lines)

4187

In [4]:
del lines[0:13]
data_string_list = []
for i in range(len(lines)):
    data_string_list.append(lines[i].split(','))
for j in range(len(data_string_list)):
    if data_string_list[j][0]=='M':
       data_string_list[j][0] = 1
    else:
        data_string_list[j][0] = 0

In [5]:
len(data_string_list)

4174

In [6]:
labels=[]
for i in range(len(data_string_list)):
    if data_string_list[i][-1] == 'negative':
        labels.append(0)
    else:
        labels.append(1)
labels=np.array(labels)

In [7]:
np.unique(labels,return_counts=True)

(array([0, 1]), array([4142,   32], dtype=int64))

In [8]:
data_string_list=np.array(data_string_list)
data_string_list=data_string_list[:,:-1]

In [9]:
data_list=[]
for i in range(len(data_string_list)):
    row_list=[]
    for j in range(len(data_string_list[0])):
        f= float(data_string_list[i][j])
        row_list.append(f)
    data_list.append(row_list)
data = np.array(data_list)

In [10]:
data.shape

(4174, 8)

### K-fold:

In [12]:
def get_folds(data,labels,k):
  np.random.seed(42)
  shuffle=np.random.permutation(len(data))
  data=data[shuffle]
  labels=labels[shuffle]

  block_length=int(len(data)/k)
  data_folds=[]
  label_folds=[]
  for i in range(k):
    if i==(k-1):
      data_fold=data[i*block_length:]
      data_folds.append(data_fold)
      label_fold=labels[i*block_length:]
      label_folds.append(label_fold)
    else:
      data_fold=data[i*block_length:block_length*(i+1)]
      data_folds.append(data_fold)
      label_fold=labels[i*block_length:block_length*(i+1)]
      label_folds.append(label_fold)
  return np.array(data_folds,dtype=object),np.array(label_folds,dtype=object)

In [13]:
def divide_data_into_k_folds(data,labels,k):
  data_folds_array,label_folds_array=get_folds(data,labels,k)
  all_data_folds_list=[]
  all_labels_folds_list=[]
  for i in range(k):
    data_fold_k=[np.concatenate(data_folds_array[np.where(np.array(list(range(k)))!=i)[0]]), data_folds_array[i]]
    all_data_folds_list.append(data_fold_k)

    label_fold_k=[np.concatenate(label_folds_array[np.where(np.array(list(range(k)))!=i)[0]]), label_folds_array[i]]
    all_labels_folds_list.append(label_fold_k)

  return all_data_folds_list, all_labels_folds_list

In [14]:
all_data_folds_list, all_labels_folds_list=divide_data_into_k_folds(data,labels,5)

In [15]:
X_train,X_test=all_data_folds_list[0][0],all_data_folds_list[0][1]

In [16]:
X_train.shape

(3340, 8)

In [17]:
y_train,y_test=all_labels_folds_list[0][0],all_labels_folds_list[0][1]

In [18]:
X_train_minority=[]
for i in np.where(y_train==1):
    a=X_train[i]
    X_train_minority.append(a)

In [19]:
X_train_minority_class=np.array(X_train_minority[0])

In [20]:
X_train_minority_class.shape

(28, 8)

In [21]:
X_train_majority=[]
for i in np.where(y_train==0):
    a=X_train[i]
    X_train_majority.append(a)

In [22]:
X_train_majority_class=np.array(X_train_majority[0])

In [23]:
X_train_majority_class.shape

(3312, 8)

### Kernel Density Functon:

In [25]:
def k(H,x,d):
  H_inv = np.linalg.inv(H)
  x_tras=np.matrix.transpose(x)
  h= np.linalg.det(H)
  m=np.matmul(x_tras,H_inv)
  m1=np.matmul(m,x)
  e=np.exp(-0.5*m1)
  n=2*(np.pi)**(-d/2)
  v=n*(h)**(-1/2)*(e)
  return v

In [26]:
def density_gen(X, X_minority):
    density_list=[]
    for j in range(len(X)):
        s=0
        for i in range(len(X_minority)):
            xi=X_minority[i]
            dif=X[j]-xi
            s=s+k(np.identity(X_minority.shape[1]), dif, X_minority.shape[1])
        density_list.append(s/len(X_minority))
    return np.array(density_list)

In [27]:
unique_labels,counts=np.unique(y_train, return_counts=True)

In [28]:
def metropolis_hastings(target_density, X_minority, size=100):
    np.random.seed(43)
    burnin_size = 10000
    size += burnin_size
    x0 =X_minority[np.random.choice(list(range(len(X_train_minority_class))),1)]
    xt = x0
    samples = []
    for i in tqdm(range(size)):
        xt_candidate = np.array([np.random.multivariate_normal(xt[0], 0.2*np.eye(X_minority.shape[1]))])
        accept_odds = (target_density(xt_candidate,X_minority))/(target_density(xt,X_minority))
        if np.random.uniform(0, 1) < accept_odds:
            xt = xt_candidate
        samples.append(xt)
    samples = np.array(samples[burnin_size:])
    samples = np.reshape(samples, [samples.shape[0], X_minority.shape[1]])
    return samples

In [29]:
import smote_variants as sv

oversampler_SMOTE=sv.SMOTE()
oversampler_polynome_fit = sv.polynom_fit_SMOTE_star()
oversampler_ProWSyn= sv.ProWSyn()
oversampler_SMOTE_IPF= sv.SMOTE_IPF()
oversampler_Lee= sv.Lee()


In [30]:
oversampler_object_list = [oversampler_SMOTE,oversampler_polynome_fit,oversampler_ProWSyn, oversampler_Lee, metropolis_hastings]

In [31]:
oversampler_name_list = ['SMOTE','polynome_fit_SMOTE','ProWSyn', 'Lee', 'Metropolis_hastings']

In [32]:
def get_imb_oversampling(X_train,y_train,X_test,y_test,oversampler_index,classifier,visualize=True):
  ## MH is in the end of the list oversampler_object_list
  if oversampler_index!=len(oversampler_object_list)-1:
    X_samp, y_samp = oversampler_object_list[oversampler_index].sample(X_train, y_train)

  else:
    X_train_minority=[]
    for i in np.where(y_train==1):
      a=X_train[i]
      X_train_minority.append(a)

    X_train_majority=[]
    for i in np.where(y_train==0):
      a=X_train[i]
      X_train_majority.append(a)
    X_train_minority_class=np.array(X_train_minority[0])
    X_train_majority_class=np.array(X_train_majority[0])

    unique_labels,counts=np.unique(y_train, return_counts=True)

    synthetic_minority_data = metropolis_hastings(density_gen, X_train_minority_class, size=counts[0]-counts[1])
    balanced_minority=np.concatenate([X_train_minority_class, synthetic_minority_data])
    balanced_data=np.concatenate([X_train_majority_class,balanced_minority])
    balanced_labels_for_classification=np.concatenate([np.zeros(len(X_train_majority_class)),np.ones(len(balanced_minority))])
    X_samp=balanced_data
    y_samp=balanced_labels_for_classification

  classifier.fit(X_samp,y_samp)
  y_pred=classifier.predict(X_test)
  cm=confusion_matrix(y_test, y_pred)
  acc=accuracy_score(y_test, y_pred)
  b_acc=balanced_accuracy_score(y_test, y_pred)
  gm=geometric_mean_score(y_test, y_pred,average='micro')
  f1=f1_score(y_test, y_pred,average='micro')
  ck=cohen_kappa_score(y_test, y_pred)

  if visualize==True:
    scaler = StandardScaler()
    scaler.fit(X_samp)
    scaled_X_samp = scaler.transform(X_samp)
    pca = PCA(n_components =2)
    X_samp_pca=pca.fit_transform(scaled_X_samp)

    plt.figure(figsize=(8,6))
    color=["r","b"]
    color_counter=0
    for i in [-1,1]:
      plt.scatter(X_samp_pca[:,0][np.where(y_samp==i)],X_samp_pca[:,1][np.where(y_samp==i)], s=10, c=color[color_counter])
      color_counter=color_counter+1
    plt.legend(['Majority','Manority'],fontsize=10, loc='lower right')
    plt.xlabel("Principal Component 1",fontsize=12)
    plt.ylabel("Principlal Component 2 ",fontsize=12)
    plt.title("PCA plot of majority class + minority class using"+""+ oversampler_name_list[oversampler_index]+"technique",fontsize=14)


  return [cm, acc,b_acc,gm, f1, ck]

In [33]:
def calc_avg_performance(performance_index,results_all_folds):
  ## performance index takes the index of the performance measure foe which we want to average over folds
  ## results_all_folds are the results of a particular oversampler run over all folds recording all performance measures
  s=0
  for result in results_all_folds:
    s=s+result[performance_index]
  avg=s/len(results_all_folds)

  s=0
  for result in results_all_folds:
    s=s+(result[performance_index]-avg)**2
  std=np.sqrt(s/len(results_all_folds))
  return avg, std

In [34]:
clf= svm.SVC()

In [35]:
perfromance_measures=['accuracy','balanced accuracy','g-mean','f1-score','cohen_kappa']
def get_results_all_folds(oversampler_index,classifier, visualize=False, get_detailed_results=False):

  results_all_folds=[]
  for i in range(len(all_data_folds_list)):
    X_train,X_test=all_data_folds_list[i][0],all_data_folds_list[i][1]
    y_train,y_test=all_labels_folds_list[i][0],all_labels_folds_list[i][1]
    results_single_fold=get_imb_oversampling(X_train,y_train,X_test,y_test,oversampler_index,clf,visualize=visualize)
    results_all_folds.append(results_single_fold)

  performance_summary_oversampler_classifer={}
  for performance_index in range(1,6):
    avg_std=calc_avg_performance(performance_index,results_all_folds)
    performance_summary_oversampler_classifer[perfromance_measures[performance_index-1]]=avg_std

  if get_detailed_results==True:
    return results_all_folds, performance_summary_oversampler_classifer
  else:
    return performance_summary_oversampler_classifer

In [36]:
get_results_all_folds(0,clf)

2024-08-23 12:29:09,078:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-23 12:29:09,082:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-23 12:29:09,082:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2024-08-23 12:29:09,082:INFO:SMOTE: simplex sampling with n_dim 2
2024-08-23 12:29:09,849:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-23 12:29:09,849:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-23 12:29:09,849:INFO:NearestNeighborsW

{'accuracy': (0.8049804547769899, 0.010989901104878694),
 'balanced accuracy': (0.7428275530899786, 0.1508620691311719),
 'g-mean': (0.8049804547769899, 0.010989901104878694),
 'f1-score': (0.8049804547769899, 0.010989901104878694),
 'cohen_kappa': (0.03840238017285695, 0.028661873577120217)}

In [37]:
summary_performace_for_all_oversampling_algorithms_all_folds=[]
for i in range(len(oversampler_object_list)):
  summary_performace_for_all_oversampling_algorithms_all_folds.append(get_results_all_folds(i,clf))

2024-08-23 12:29:12,980:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-23 12:29:12,982:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-23 12:29:12,983:INFO:NearestNeighborsWithMetricTensor: kneighbors query minkowski
2024-08-23 12:29:12,984:INFO:SMOTE: simplex sampling with n_dim 2
2024-08-23 12:29:13,770:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'nn_params': {}, 'n_jobs': 1, 'ss_params': {'n_dim': 2, 'simplex_sampling': 'random', 'within_simplex_sampling': 'random', 'gaussian_component': {}}, 'random_state': None, 'class_name': 'SMOTE'}")
2024-08-23 12:29:13,771:INFO:NearestNeighborsWithMetricTensor: NN fitting with metric minkowski
2024-08-23 12:29:13,772:INFO:NearestNeighborsW

  0%|          | 0/13284 [00:00<?, ?it/s]

  0%|          | 0/13286 [00:00<?, ?it/s]

  0%|          | 0/13296 [00:00<?, ?it/s]

  0%|          | 0/13294 [00:00<?, ?it/s]

  0%|          | 0/13280 [00:00<?, ?it/s]

In [38]:
summary_performace_for_all_oversampling_algorithms_all_folds_list=np.array([[[round(i,3) for i in summary_performace_for_all_oversampling_algorithms_all_folds[oversampler_index][key]] for key in summary_performace_for_all_oversampling_algorithms_all_folds[0].keys()] for oversampler_index in range(len(oversampler_name_list))])

In [39]:
oversamplers_result_summary_list=[]
for i in range(summary_performace_for_all_oversampling_algorithms_all_folds_list.shape[0]):
  performance_results_summary_list=[]
  for j in range(summary_performace_for_all_oversampling_algorithms_all_folds_list.shape[1]):
    performance_results_summary_list.append(str(summary_performace_for_all_oversampling_algorithms_all_folds_list[i][j][0])+"\u00B1"+str(summary_performace_for_all_oversampling_algorithms_all_folds_list[i][j][1]))
  oversamplers_result_summary_list.append(performance_results_summary_list)
oversamplers_result_summary_list=np.array(oversamplers_result_summary_list)

In [40]:
performance_results_summary_list

['0.992±0.003', '0.5±0.0', '0.992±0.003', '0.992±0.003', '-0.0±0.001']

In [41]:
oversamplers_result_summary_list

array([['0.805±0.014', '0.718±0.131', '0.805±0.014', '0.805±0.014',
        '0.036±0.029'],
       ['0.992±0.003', '0.5±0.0', '0.992±0.003', '0.992±0.003',
        '0.0±0.0'],
       ['0.866±0.011', '0.692±0.09', '0.866±0.011', '0.866±0.011',
        '0.044±0.03'],
       ['0.948±0.042', '0.602±0.075', '0.948±0.042', '0.948±0.042',
        '0.062±0.049'],
       ['0.992±0.003', '0.5±0.0', '0.992±0.003', '0.992±0.003',
        '-0.0±0.001']], dtype='<U11')

In [42]:
df=pd.DataFrame(oversamplers_result_summary_list,columns=perfromance_measures,index=oversampler_name_list)

In [43]:
df

,accuracy,balanced accuracy,g-mean,f1-score,cohen_kappa
SMOTE,0.805±0.014,0.718±0.131,0.805±0.014,0.805±0.014,0.036±0.029
polynome_fit_SMOTE,0.992±0.003,0.5±0.0,0.992±0.003,0.992±0.003,0.0±0.0
ProWSyn,0.866±0.011,0.692±0.09,0.866±0.011,0.866±0.011,0.044±0.03
Lee,0.948±0.042,0.602±0.075,0.948±0.042,0.948±0.042,0.062±0.049
Metropolis_hastings,0.992±0.003,0.5±0.0,0.992±0.003,0.992±0.003,-0.0±0.001


In [44]:
with pd.ExcelWriter('SVM_clf_result_for_d5.xlsx') as excel_writer:
  df.to_excel(excel_writer,sheet_name='sheet_1')